<a href="https://colab.research.google.com/github/GerardoMunoz/embedded/blob/main/CoAP_MQTT_AMQP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MQTT vs CoAP and AMQP

## 1. CoAP (Constrained Application Protocol)

![](https://raw.githubusercontent.com/GerardoMunoz/embedded/refs/heads/main/imgs/CoAP.PNG)

* Type: Request/response protocol similar to HTTP, but optimized for IoT.
* Transport: Runs over UDP (also supports DTLS for security).
* Strengths:
  * Very lightweight, designed for constrained nodes/networks.
  * Maps closely to REST (GET, PUT, POST, DELETE).
  * Supports multicast → efficient for group communication.
* Weaknesses:
  * Less mature ecosystem than MQTT.
  * UDP-based → packet loss more likely unless reliability mechanisms are used.
* Use case: Smart lighting, constrained embedded systems, M2M communication.
* Protocol overhead: Extremely small.
  * Basic CoAP header = 4 bytes + options.
* Payload max size:
  * Limited by underlying UDP datagram.
  * Usually ≤ 64 KB, but in practice constrained networks (6LoWPAN) limit to < 1.5 KB.
  * Larger payloads require block-wise transfer (fragmentation at CoAP level).
* Best for: Very small state/config updates (a few bytes).



## 2. MQTT (Message Queuing Telemetry Transport)

![](https://raw.githubusercontent.com/GerardoMunoz/embedded/refs/heads/main/imgs/MQTT.PNG)

*Type: Lightweight publish/subscribe messaging protocol.
* Transport: Runs over TCP (sometimes over TLS for security).
* Strengths:
  * Very lightweight → good for constrained devices.
  * Uses broker-based model (clients publish/subscribe to topics).
  * Reliable QoS levels (0 = at most once, 1 = at least once, 2 = exactly once).
* Weaknesses:
  * Needs a central broker → single point of failure unless clustered.
  * TCP overhead can be high for extremely constrained devices.
  * Use case: Sensor networks, telemetry, home automation.
* Protocol overhead: Very small (as low as 2 bytes for fixed header).
* Payload max size:
  * The spec allows up to 256 MB.
  * In practice, brokers/devices often restrict this (commonly 256 KB – 1 MB).
* Best for: Small telemetry messages (a few bytes to a few KB).

3. AMQP (Advanced Message Queuing Protocol)
![](https://raw.githubusercontent.com/GerardoMunoz/embedded/refs/heads/main/imgs/AMQP.PNG)

* Type: Full-featured messaging protocol with queues, routing, transactions.
* Transport: TCP (TLS possible).
* Strengths:
  * Rich message semantics (routing, queuing, acknowledgments).
  * Enterprise-grade → reliable and secure.
  * Decouples producers and consumers (like MQTT, but more powerful).
* Weaknesses:
  * Heavyweight compared to MQTT/CoAP.
  * Higher resource usage → not suitable for tiny IoT devices.
* Use case: Financial services, enterprise IoT backends, cloud messaging.
* Protocol overhead: Much larger (framing, headers, properties).
  * Typically dozens to hundreds of bytes per message before payload.
* Payload max size:
  * Spec allows very large messages (up to 2^32−1 bytes ≈ 4 GB).
  * Actual broker limits vary (RabbitMQ default is 128 MB).
* Best for: Enterprise use cases → can handle KB to MB-scale messages.


# MQTT


## What is MQTT?
- MQTT = Message Queuing Telemetry Transport
- Lightweight publish/subscribe protocol
- Designed for low-bandwidth, high-latency networks
- Common in IoT, robotics, and mobile apps

## MQTT Basics
- **Broker**: central server managing messages
- **Client**: device or app publishing/subscribing
- **Topics**: hierarchical channels for messages
- **Messages**: payloads published to topics
  - Retained messages
  - Last Will and Testament (LWT)
- **Security**: TLS (such as HTTPS)

## Topic Structure
- Hierarchical, separated by `/`
- Example: `<identifier>/<especie>/<medición>/<sensorId>`
  - `ranger/cherry_tree/soil/sensor1`
  - `ranger/pine/air/sensor3`
- Wildcards:
  - `+` matches one level `sensors/+/soil/+`
  - `#` matches multiple levels `sensors/pine/#`

## Quality of Service (QoS)
- QoS 0: At most once (fast, no guarantee)
- QoS 1: At least once (possible duplicates)
- QoS 2: Exactly once (reliable, slower)


## Versions & Timeline
- 1999: MQTT 1.0 (IBM/Arcom)
- 2010: MQTT 3.1 released
- 2014: MQTT 3.1.1 (OASIS, ISO/IEC 20922)
- 2019: MQTT 5.0 adds advanced features

## MQTT in IoT
- Sensors publish data (temperature, humidity, etc.)
- Robots receive commands via topics
- Flexible topic design enables filtering and grouping
- Common in smart homes, agriculture, industry

## Open-Source Brokers
- **Mosquitto**: lightweight, popular for prototyping
- **EMQX**: scalable, MQTT 5 support
- **NanoMQ**: optimized for edge
- HiveMQ: commercial, with cloud service

## Client Libraries
- Python: Paho, gmqtt
- JavaScript: mqtt.js
- C/C++: libmosquitto
- Java: Paho Java, HiveMQ client
- C#: MQTTnet



## What is mqtt.js?
- A JavaScript library for connecting to **MQTT** brokers.
- Works in:
  - **Browsers** (via WebSocket).
  - **Node.js** (TCP, TLS, WebSocket).
- Compatible with **MQTT 3.1.1** and **MQTT 5.0**.



### Installation in Browser (HTML)
```html
<script src="https://cdn.jsdelivr.net/npm/mqtt/dist/mqtt.min.js"></script>
```



### Connecting to a broker
```js
import mqtt from "mqtt";

// Example for Node.js
const client = mqtt.connect("mqtt://broker.hivemq.com", {
  clientId: "js-client-" + Math.random().toString(16).substr(2, 8),
  clean: true,
  connectTimeout: 4000,
  reconnectPeriod: 1000,
});

// Events
client.on("connect", () => {
  console.log("✅ Connected to broker");
});

client.on("error", (err) => {
  console.error("❌ Connection error:", err);
});
```


### Subscribing to topics
```js
client.subscribe("sensors/+/temperature", { qos: 1 }, (err, granted) => {
  if (err) console.error("Subscription error:", err);
  else console.log("Subscribed to:", granted.map(g => g.topic).join(", "));
});

// Receiving messages
client.on("message", (topic, payload) => {
  console.log("📩 Message on", topic, ":", payload.toString());
});
```



### Publishing messages
```js
const data = { temperature: 23.5, unit: "C" };

client.publish("sensors/lab/temperature", JSON.stringify(data), {
  qos: 1,
  retain: false,
}, (err) => {
  if (err) console.error("Publish error:", err);
  else console.log("✅ Message published");
});
```



### Key Options
- **clientId**: unique client identifier.
- **clean**: if `false`, broker stores session (subscriptions).
- **qos**:
  - 0: at most once.
  - 1: at least once (with ACK).
  - 2: exactly once.
- **retain**: broker keeps the last published message.
- **reconnectPeriod**: time between reconnection attempts.


### MQTT 5.0 in mqtt.js
Enable with:
```js
const client = mqtt.connect("ws://broker:port", {
  protocolVersion: 5,
});
```
Enables:
- Message properties.
- Subscription reason codes.
- Session expiry.


### Full Example (Browser)
```html
<script src="https://cdn.jsdelivr.net/npm/mqtt/dist/mqtt.min.js"></script>
<script>
  const client = mqtt.connect("wss://broker.hivemq.com:8884/mqtt");

  client.on("connect", () => {
    console.log("Connected ✅");
    client.subscribe("demo/test");
    client.publish("demo/test", "Hello from the browser!");
  });

  client.on("message", (topic, msg) => {
    console.log("Message on", topic, ":", msg.toString());
  });
</script>
```


### Resources
- [Official Repository](https://github.com/mqttjs/MQTT.js)
- [API Documentation](https://github.com/mqttjs/MQTT.js#api)
- Public brokers for testing:
  - `test.mosquitto.org`
  - `broker.hivemq.com`
  - `mqtt.eclipseprojects.io`


## Web example
https://raw.githubusercontent.com/GerardoMunoz/embedded/refs/heads/main/mqtt-demo.html



```HTML
<!doctype html>
<html lang="es">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>Demo MQTT (WebSocket) — Enviar y recibir</title>
  <style>
    body { font-family: Inter, system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", Arial; margin: 18px; color: #111; }
    h1 { font-size: 20px; margin-bottom: 6px; }
    p.lead { margin-top: 0; margin-bottom: 12px; color: #444; }
    label { display:block; margin-top:10px; font-weight:600; font-size:13px; }
    input[type=text], input[type=url], select, textarea {
      width: 100%; padding:8px 10px; margin-top:6px; box-sizing:border-box; border-radius:6px; border:1px solid #ddd;
    }
    button { padding:8px 12px; border-radius:6px; border:none; background:#0066cc; color:white; cursor:pointer; }
    button.secondary { background:#444; }
    .row { display:flex; gap:12px; }
    .col { flex:1; }
    #log { height:260px; overflow:auto; border:1px solid #eee; padding:8px; background:#fafafa; border-radius:6px; white-space:pre-wrap; font-family:monospace; font-size:13px; }
    .controls { display:flex; gap:8px; margin-top:12px; align-items:center; flex-wrap:wrap }
    small.hint { color:#666; }
    footer { margin-top:18px; color:#666; font-size:13px; }
    .status { display:inline-block; padding:4px 8px; border-radius:999px; background:#eee; font-weight:600; }
    .status.connected { background: #dff0d8; color:#2b6a2b; }
    .status.disconnected { background:#fde6e6; color:#7a1a1a; }
    .flex-between { display:flex; justify-content:space-between; align-items:center; gap:12px; }
    .small-input { width:120px; }
  </style>
</head>
<body>
  <h1>Demo MQTT (WebSocket) — Enviar y recibir</h1>
  <p class="lead">Interfaz simple para conectarte a un broker MQTT vía WebSocket, suscribirte y publicar. Reemplaza la URL del broker por la que uses.</p>

  <label>Broker WebSocket URL (ejemplos):
    <input id="brokerUrl" type="url" value="wss://broker.hivemq.com:8884/mqtt" />
    <small class="hint">Ejemplos: <code>ws://test.mosquitto.org:8080</code> (no TLS), <code>wss://broker.hivemq.com:8884/mqtt</code> (TLS). Verifica el endpoint WebSocket del broker.</small>
  </label>

  <div class="row">
    <div class="col">
      <label>Client ID
        <input id="clientId" type="text" value="webclient-<?= Math.floor(Math.random()*10000) ?>" />
      </label>
    </div>
    <div class="col">
      <label>Username (opcional)
        <input id="username" type="text" placeholder="usuario (opcional)" />
      </label>
    </div>
    <div class="col">
      <label>Password (opcional)
        <input id="password" type="text" placeholder="contraseña (opcional)" />
      </label>
    </div>
  </div>

  <div class="controls">
    <button id="connectBtn">Conectar</button>
    <button id="disconnectBtn" class="secondary" disabled>Desconectar</button>
    <div style="margin-left:auto;">
      Estado: <span id="statusBadge" class="status disconnected">desconectado</span>
    </div>
  </div>

  <hr />

  <div class="flex-between">
    <div style="flex:1; margin-right:12px;">
      <label>Suscribirse a topic
        <input id="subTopic" type="text" value="sensores/+/suelo/+" />
      </label>
      <div style="margin-top:8px;">
        <button id="subscribeBtn" class="secondary" disabled>Suscribir</button>
        <button id="unsubscribeBtn" class="secondary" disabled>Desuscribir</button>
      </div>
    </div>

    <div style="width:340px;">
      <label>Publicar mensaje
        <input id="pubTopic" type="text" value="sensores/roble/suelo/sensor1" />
      </label>
      <label>Payload (texto o JSON)
        <textarea id="pubPayload" rows="3">{ "humedad": 42 }</textarea>
      </label>
      <div style="display:flex; gap:8px; align-items:center; margin-top:6px;">
        <label style="margin:0;">QoS
          <select id="pubQos" class="small-input">
            <option value="0">0</option>
            <option value="1">1</option>
            <option value="2">2</option>
          </select>
        </label>
        <label style="margin:0;">Retain
          <select id="pubRetain" class="small-input">
            <option value="false">false</option>
            <option value="true">true</option>
          </select>
        </label>

        <button id="publishBtn">Publicar</button>
      </div>
    </div>
  </div>

  <h3 style="margin-top:16px;">Registro de mensajes</h3>
  <div id="log"></div>

  <footer>
    <p>Notas: asegúrate de que el broker soporte conexiones WebSocket. Si usas un broker local, habilita websocket en su configuración (p. ej. Mosquitto, EMQX, HiveMQ).</p>
  </footer>

  <!-- mqtt.js CDN -->
  <script src="https://cdn.jsdelivr.net/npm/mqtt/dist/mqtt.min.js"></script>
  <script>
    // Elementos
    const connectBtn = document.getElementById('connectBtn');
    const disconnectBtn = document.getElementById('disconnectBtn');
    const brokerUrlInput = document.getElementById('brokerUrl');
    const clientIdInput = document.getElementById('clientId');
    const usernameInput = document.getElementById('username');
    const passwordInput = document.getElementById('password');
    const statusBadge = document.getElementById('statusBadge');

    const subscribeBtn = document.getElementById('subscribeBtn');
    const unsubscribeBtn = document.getElementById('unsubscribeBtn');
    const subTopicInput = document.getElementById('subTopic');

    const publishBtn = document.getElementById('publishBtn');
    const pubTopicInput = document.getElementById('pubTopic');
    const pubPayloadInput = document.getElementById('pubPayload');
    const pubQosInput = document.getElementById('pubQos');
    const pubRetainInput = document.getElementById('pubRetain');

    const logEl = document.getElementById('log');

    let client = null;
    let currentSubscriptions = new Set();

    function log(msg, type='info') {
      const time = new Date().toLocaleTimeString();
      logEl.textContent = `[${time}] ${msg}\n` + logEl.textContent;
    }

    function setStatus(connected) {
      if (connected) {
        statusBadge.textContent = 'conectado';
        statusBadge.classList.remove('disconnected');
        statusBadge.classList.add('connected');
      } else {
        statusBadge.textContent = 'desconectado';
        statusBadge.classList.remove('connected');
        statusBadge.classList.add('disconnected');
      }
    }

    connectBtn.addEventListener('click', () => {
      if (client && client.connected) { log('Ya estás conectado.'); return; }

      const brokerUrl = brokerUrlInput.value.trim();
      if (!brokerUrl) { alert('Especifica la URL WebSocket del broker.'); return; }

      const clientId = clientIdInput.value || 'webclient-' + Math.floor(Math.random()*10000);
      const username = usernameInput.value || undefined;
      const password = passwordInput.value || undefined;

      log(`Conectando a ${brokerUrl} con clientId=${clientId} ...`);
      // opciones básicas; puedes ajustar keepalive, clean, etc.
      const opts = {
        clientId,
        username,
        password,
        clean: true,
        reconnectPeriod: 4000, // reintento automático
        connectTimeout: 30 * 1000
      };

      try {
        client = mqtt.connect(brokerUrl, opts);
      } catch (err) {
        log('Error creando conexión: ' + err.message);
        return;
      }

      // Eventos
      client.on('connect', (connack) => {
        log('Conectado al broker. connack: ' + JSON.stringify(connack || {}));
        setStatus(true);
        connectBtn.disabled = true;
        disconnectBtn.disabled = false;
        subscribeBtn.disabled = false;
        publishBtn.disabled = false;
      });

      client.on('reconnect', () => {
        log('Intentando reconectar...');
      });

      client.on('close', () => {
        log('Conexión cerrada.');
        setStatus(false);
        connectBtn.disabled = false;
        disconnectBtn.disabled = true;
        subscribeBtn.disabled = true;
        unsubscribeBtn.disabled = true;
        publishBtn.disabled = true;
      });

      client.on('offline', () => {
        log('Cliente offline.');
      });

      client.on('error', (err) => {
        log('Error: ' + (err && err.message ? err.message : JSON.stringify(err)));
      });

      client.on('message', (topic, payload, packet) => {
        // Intentar parsear JSON si parece JSON
        let text = payload.toString();
        let parsed = null;
        try {
          parsed = JSON.parse(text);
        } catch (e) { /* no es JSON */ }
        const qos = packet && packet.qos != null ? packet.qos : '?';
        const retain = packet && packet.retain ? ' [retained]' : '';
        const display = parsed ? JSON.stringify(parsed, null, 2) : text;
        log(`<< ${topic} (qos=${qos})${retain}\n${display}`);
      });
    });

    disconnectBtn.addEventListener('click', () => {
      if (!client) return;
      log('Desconectando...');
      client.end(false, {}, () => {
        log('Desconectado (end).');
        setStatus(false);
        connectBtn.disabled = false;
        disconnectBtn.disabled = true;
        subscribeBtn.disabled = true;
        unsubscribeBtn.disabled = true;
        publishBtn.disabled = true;
      });
    });

    subscribeBtn.addEventListener('click', () => {
      if (!client || !client.connected) { alert('No conectado'); return; }
      const topic = subTopicInput.value.trim();
      if (!topic) return;
      const qos = 0;
      client.subscribe(topic, {qos}, (err, granted) => {
        if (err) {
          log('Error subscribe: ' + err.message);
          return;
        }
        log(`Suscrito a "${topic}". granted: ${JSON.stringify(granted)}`);
        currentSubscriptions.add(topic);
        unsubscribeBtn.disabled = false;
      });
    });

    unsubscribeBtn.addEventListener('click', () => {
      if (!client || !client.connected) { alert('No conectado'); return; }
      const topic = subTopicInput.value.trim();
      if (!topic) return;
      client.unsubscribe(topic, (err) => {
        if (err) { log('Error unsubscribe: ' + err.message); return; }
        log(`Desuscrito de "${topic}".`);
        currentSubscriptions.delete(topic);
        if (currentSubscriptions.size === 0) unsubscribeBtn.disabled = true;
      });
    });

    publishBtn.addEventListener('click', () => {
      if (!client || !client.connected) { alert('No conectado'); return; }
      const topic = pubTopicInput.value.trim();
      if (!topic) { alert('Especifica un topic'); return; }
      let payload = pubPayloadInput.value;
      // Si parece JSON, enviamos tal cual (string), broker no interpreta
      const qos = Number(pubQosInput.value);
      const retain = (pubRetainInput.value === 'true');
      client.publish(topic, payload, {qos, retain}, (err) => {
        if (err) log('Error publish: ' + err.message);
        else log(`>> ${topic} (qos=${qos}) ${retain ? '[retained]' : ''}\n${payload}`);
      });
    });

    // inicializar
    setStatus(false);
    // generar clientId razonable en carga
    (function initClientId() {
      const el = clientIdInput;
      if (!el.value || el.value.startsWith('webclient-<?= ')) {
        el.value = 'webclient-' + Math.floor(Math.random() * 90000 + 10000);
      }
    })();
  </script>
</body>
</html>
```